In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from md04_func import *

df=pd.DataFrame(md04['data'], columns=md04['header'])
df['cum']=df.Change.cumsum()
df.cum.plot(x="Date")
plt.plot(df.Date, df.cum)
target=df.mask('Type','ShipNt')
plt.plot(target.Date, target.cum)

filelist=[]
f=open("MD04list.txt","r", encoding="UTF-8")
list=f.readlines()
for i in list :        
    document=fileRead(i.rstrip("\n"))
    md04=parsingDoc(document)
    PD=pd.DataFrame(md04["data"], columns=md04["header"]) 
    
    
    
fig = plt.figure()
ax= PD['Rec/reqd Qty'].plot(x="Date", kind="bar", title=md04['meta'][0])
ax2 = ax.twinx()
ax2.plot(ax.get_xticks(),PD['AvailQty'])

## CONSTAT / DICTIONARY FOR PARSING

In [2]:
category = {
    'ShipNt' :  "Receipt",
    'SchLne' :  "Receipt",    
    'Stock'  :  "Receipt",
    'CStock' : "Reqmt",
    'Ordres' : "Reqmt",    
    'Order'  :  "Reqmt",
    'Deliv.' : "Reqmt",
    'PrdOrd' : "", #normally assigned together with cstock. no need to count
    'OrdRes' : "Reqmt",
    'SafeSt' : "",
    'StLcSt' : "",
    'PurRqs' : "",
    'IndReq' : ""
}
header=["Partnr","SO_Date","SO_MRP", "SO_NR", "SO_POS","SO_QTY", "PO_DATE","PO_MRP","PO_NR", "PO_POS","PO_QTY" ]
total=[]


## FUNCTION

In [3]:
def kmatParser(df) : 
    MappedResult=[]
    idx=mask(df, "Type", "CStock").index
    CStockReqmt=df.iloc[idx]
    CStockReceipt=df.iloc[idx+1]

    if len(CStockReqmt) != len(CStockReceipt) : 
        print("Please check CStock as rows are not matched")
    
    for i,val in enumerate(CStockReceipt.iterrows()):
        MappedResult.append (CStockReqmt.iloc[i].tolist()[0:6] + (CStockReceipt.iloc[i].tolist()[1:6]))
    
    return MappedResult, df.drop(index=idx+1).drop(index=idx)   

In [15]:
def fehaParser(df) :
    ReceiptQueue=[]
    ReqmtQueue=[]
    MappedResult=[]
    flag=0
    Receipt=[]
    Reqmt=[]

    
    #KMAT PARSER
    MappedResult,df = kmatParser(df)    
    
    
    for index, line in df.iterrows() :    
        print(line)
        if line[6]==0  and line[2]== 'Stock' : continue
        if line[2] == 'Stock' : 
            line[5]=line[6]
        if category[line[2]] == 'Receipt' :        
            ReceiptQueue.append(line)
        if category[line[2]] == 'Reqmt' :
            ReqmtQueue.append(line)
        else :
            continue
            
    #If df only has kmat (CStock, then it return )
    if len(df)<=1 :        
        return MappedResult    
    
    # Else it goes through the below
    else : 
        try :   
            print("DF")
            print(df,"\n")
            print("MappedResult")
            print(MappedResult, "\n\n")
            print("ReceiptQueue")
            print(ReceiptQueue, "\n\n")
            print("ReqmtQueue")
            print(ReqmtQueue, "\n\n")
            Receipt = ReceiptQueue.pop(0)
            Reqmt = ReqmtQueue.pop(0)  

            while True : 
                
                print (Receipt[5], Reqmt[5])
                if Receipt[5] + Reqmt[5] == 0 :                    
                    flag=0
                    MappedResult.append([Reqmt[0], Reqmt[1], Reqmt[2], Reqmt[3], Reqmt[4], Reqmt[5],  Receipt[1], Receipt[2], Receipt[3], Receipt[4], Receipt[5]])    
                    print("DF")
                    print(df,"\n")
                    print("ReceiptQueue")
                    print(ReceiptQueue, "\n")
                    print("ReqmtQueue")
                    print(ReqmtQueue, "\n\n")
                    
                    if len(ReceiptQueue) +  len(ReqmtQueue) <=0 : 
                        return MappedResult                        
                    
                    Receipt=[]; Receipt = ReceiptQueue.pop(0)
                    Reqmt=[]; Reqmt = ReqmtQueue.pop(0)  

                if Receipt[5] + Reqmt[5] > 0 :                           
                    flag=1
                    MappedResult.append([Reqmt[0], Reqmt[1], Reqmt[2], Reqmt[3], Reqmt[4], Reqmt[5],  Receipt[1], Receipt[2], Receipt[3], Receipt[4], -Reqmt[5]])    
                    Receipt[5] +=Reqmt[5]            
                    Reqmt = ReqmtQueue.pop(0)  


                if Receipt[5] + Reqmt[5] < 0 :                                
                    flag=-1
                    MappedResult.append([Reqmt[0], Reqmt[1], Reqmt[2], Reqmt[3], Reqmt[4], -Receipt[5],  Receipt[1], Receipt[2], Receipt[3], Receipt[4], Receipt[5]])
                    Reqmt[5] +=Receipt[5]
                    Receipt = ReceiptQueue.pop(0)

            #if no elment for popping.  
        except Exception as e:   

            if flag == -1:
                MappedResult.append([Reqmt[0], Reqmt[1], Reqmt[2], Reqmt[3], Reqmt[4], Reqmt[5],'','' ,'' ,'' ,'' ]) 
            elif flag == 1:
                MappedResult.append([Receipt[0], '','','','','',  Receipt[1], Receipt[2], Receipt[3], Receipt[4], Receipt[5]])          

            elif flag == 0:
                print("flag 0")
                print("Reqmt : ")
                print(Reqmt,"\n")
                
                print("Receipt : ")
                print(Receipt, "\n")
                
                
                if len(Reqmt) == 0 and len(Receipt)==0 :                     
                    print("both empty")
                elif len(Reqmt) == 0 and len(Receipt)!=0: 
                    #print("Reqmt empty")
                    MappedResult.append([Receipt[0], '','','','','',  Receipt[1], Receipt[2], Receipt[3], Receipt[4], Receipt[5]])          
                elif len(Reqmt) !=0 and len(Receipt) ==0 :                
                    #print("Receipt empty")
                    MappedResult.append([Reqmt[0], Reqmt[1], Reqmt[2], Reqmt[3], Reqmt[4], Reqmt[5],'','' ,'' ,'' ,'' ]) 

            #print("ReqmtQueue :", len(ReqmtQueue), "ReceiptQueue :", len(ReceiptQueue))
            for Reqmt in ReqmtQueue :
                MappedResult.append([Reqmt[0], Reqmt[1], Reqmt[2], Reqmt[3], Reqmt[4], Reqmt[5],'','' ,'' ,'' ,'' ]) 

            for Receipt in ReceiptQueue :
                MappedResult.append([Receipt[0], '','','','','',  Receipt[1], Receipt[2], Receipt[3], Receipt[4], Receipt[5]])          
            print ("!!!!!!!!!!!!!!!!!!!!!!!!")
            print (MappedResult)
            print ("!!!!!!!!!!!!!!!!!!!!!!!!")
            return MappedResult

## UNIT TEST

In [17]:
total=[]
filelist=[]
md04=parsingDoc(fileRead("19145"))
PD=pd.DataFrame(md04["data"], columns=md04["header"])   
df=PD.iloc[:,0:7]    
total.append(pd.DataFrame(fehaParser(df), columns=header))

['19145', '2018/05/11', 'Stock', 0, 0, 0, 111]
['19145', '2018/05/11', 'SafeSt', 0, 0, -31, 80]
['19145', '2018/05/10', 'OrdRes', 0, 0, -15, 65]
['19145', '2018/05/11', 'Order', '5236289000', '000060', -5, 60]
['19145', '2018/05/15', 'Order', '5236289862', '000010', -1, 59]
['19145', '2018/05/15', 'OrdRes', 0, 0, -25, 34]
['19145', '2018/05/16', 'Order', '5236290321', '000060', -4, 30]
['19145', '2018/05/17', 'Order', '5236289007', '000400', -5, 25]
['19145', '2018/05/31', 'OrdRes', 0, 0, -10, 15]
['19145', '2018/06/11', 'PurRqs', '4324039195', '00010 *', 147, 162]
['19145', '2018/06/11', 'PurRqs', '4324045413', '00010 *', 147, 309]
['19145', '2018/08/06', 'PurRqs', '4324034632', '00010 *', 147, 456]
['19145', '2018/08/06', 'PurRqs', '4324045322', '00010 *', 147, 603]
['19145', '2018/05/11', 'StLcSt', 0, 0, 0, 0]
Partnr         19145
Date      2018/05/11
Type           Stock
DocNum             0
Pos                0
Change             0
Avail            111
Name: 0, dtype: object
Partn

In [18]:
total

[  Partnr     SO_Date  SO_MRP       SO_NR  SO_POS SO_QTY     PO_DATE PO_MRP  \
 0  19145  2018/05/10  OrdRes           0       0    -15  2018/05/11  Stock   
 1  19145  2018/05/11   Order  5236289000  000060     -5  2018/05/11  Stock   
 2  19145  2018/05/15   Order  5236289862  000010     -1  2018/05/11  Stock   
 3  19145  2018/05/15  OrdRes           0       0    -25  2018/05/11  Stock   
 4  19145  2018/05/16   Order  5236290321  000060     -4  2018/05/11  Stock   
 5  19145  2018/05/17   Order  5236289007  000400     -5  2018/05/11  Stock   
 6  19145  2018/05/31  OrdRes           0       0    -10  2018/05/11  Stock   
 7  19145                                                 2018/05/11  Stock   
 
    PO_NR  PO_POS  PO_QTY  
 0      0       0      15  
 1      0       0       5  
 2      0       0       1  
 3      0       0      25  
 4      0       0       4  
 5      0       0       5  
 6      0       0      10  
 7      0       0      46  ]

In [19]:
pd.DataFrame(total[0])

,Partnr,SO_Date,SO_MRP,SO_NR,SO_POS,SO_QTY,PO_DATE,PO_MRP,PO_NR,PO_POS,PO_QTY
0,19145,2018/05/10,OrdRes,0,0,-15,2018/05/11,Stock,0,0,15
1,19145,2018/05/11,Order,5236289000,000060,-5,2018/05/11,Stock,0,0,5
2,19145,2018/05/15,Order,5236289862,000010,-1,2018/05/11,Stock,0,0,1
3,19145,2018/05/15,OrdRes,0,0,-25,2018/05/11,Stock,0,0,25
4,19145,2018/05/16,Order,5236290321,000060,-4,2018/05/11,Stock,0,0,4
5,19145,2018/05/17,Order,5236289007,000400,-5,2018/05/11,Stock,0,0,5
6,19145,2018/05/31,OrdRes,0,0,-10,2018/05/11,Stock,0,0,10
7,19145,,,,,,2018/05/11,Stock,0,0,46


## ACTUAL RUNNING (for MD04list.txt)

In [21]:
total=[]
filelist=[]
f=open("MD04list.txt","r", encoding="UTF-8")
list=f.readlines()
for i in list :      
    md04=parsingDoc(fileRead(i.replace("\n","")))    
    PD=pd.DataFrame(md04["data"], columns=md04["header"])   
    #PD.plot(x="Date", kind="bar", title=md04['meta'][0])
    df=PD.iloc[:,0:7]    
    total.append(pd.DataFrame(fehaParser(df), columns=header))

['543279', '2018/05/11', 'Stock', 0, 0, 0, 55]
['543279', '2018/05/11', 'Order', '5236280796', '000010', -91, -36]
['543279', '2018/05/14', 'ShipNt', '4045488462', '00001', 36, 0]
Partnr        543279
Date      2018/05/11
Type           Stock
DocNum             0
Pos                0
Change             0
Avail             55
Name: 0, dtype: object
Partnr        543279
Date      2018/05/11
Type           Order
DocNum    5236280796
Pos           000010
Change           -91
Avail            -36
Name: 1, dtype: object
Partnr        543279
Date      2018/05/14
Type          ShipNt
DocNum    4045488462
Pos            00001
Change            36
Avail              0
Name: 2, dtype: object
DF
   Partnr        Date    Type      DocNum     Pos  Change  Avail
0  543279  2018/05/11   Stock           0       0       0     55
1  543279  2018/05/11   Order  5236280796  000010     -91    -36
2  543279  2018/05/14  ShipNt  4045488462   00001      36      0 

MappedResult
[] 


ReceiptQueue
[Partnr      

Partnr        544038
Date      2018/06/01
Type          ShipNt
DocNum    4045503748
Pos            00001
Change             1
Avail              1
Name: 4, dtype: object 

!!!!!!!!!!!!!!!!!!!!!!!!
[['544038', '2018/05/11', 'CStock', '5236285597', '000010', 0, '2018/06/04', 'ShipNt', '4045510408', '00001', 1], ['544038', '2018/05/11', 'Order', '5236282752', '000160', -1, '2018/05/17', 'SchLne', '4015506339', '00033', 1], ['544038', '2018/05/24', 'Order', '5236289739', '000010', -1, '2018/05/17', 'SchLne', '4015506339', '00033', 1], ['544038', '', '', '', '', '', '2018/06/01', 'ShipNt', '4045503748', '00001', 1]]
!!!!!!!!!!!!!!!!!!!!!!!!
['544053', '2018/05/11', 'Stock', 0, 0, 0, 0]
['544053', '2018/05/11', 'SafeSt', 0, 0, -2, -2]
['544053', '2018/05/11', 'Order', '5236282758', '000890', -1, -3]
['544053', '2018/05/11', 'Order', '5236282758', '001050', -1, -4]
['544053', '2018/05/11', 'Order', '5236282758', '001280', -1, -5]
['544053', '2018/05/23', 'PurRqs', '4324032967', '00010 *', 2, 

      Partnr        Date    Type        DocNum   Pos  Change  Avail
0   23109748  2018/05/11   Stock             0     0       0      0
3   23109748  2018/05/28  PrdOrd  009331003298  YP         1      1
6   23109748  2018/05/28  PrdOrd  009331003328  YP         2      2
9   23109748  2018/05/28  PrdOrd  009331003331  YP         1      1
12  23109748  2018/05/28  PrdOrd  009331003334  YP         1      1
15  23109748  2018/05/28  PrdOrd  009331003335  YP         1      1
18  23109748  2018/05/28  PrdOrd  009331003336  YP         1      1
21  23109748  2018/05/28  PrdOrd  009331003337  YP         1      1
24  23109748  2018/05/28  PrdOrd  009331003338  YP         1      1
27  23109748  2018/05/28  PrdOrd  009331003339  YP         1      1
30  23109748  2018/05/28  PrdOrd  009331003341  YP         1      1
33  23109748  2018/05/28  PrdOrd  009331003342  YP         1      1
36  23109748  2018/05/28  PrdOrd  009331003346  YP         1      1
39  23109748  2018/05/28  PrdOrd  009331003348  

Partnr        544011
Date      2018/05/11
Type           Stock
DocNum             0
Pos                0
Change             0
Avail              1
Name: 0, dtype: object
Partnr        544011
Date      2018/05/11
Type          SafeSt
DocNum             0
Pos                0
Change            -3
Avail             -2
Name: 1, dtype: object
Partnr        544011
Date      2018/05/17
Type          SchLne
DocNum    4015506339
Pos            00027
Change             2
Avail              0
Name: 2, dtype: object
Partnr        544011
Date      2018/05/23
Type          PurRqs
DocNum    4324043502
Pos          00010 *
Change             3
Avail              3
Name: 3, dtype: object
Partnr        544011
Date      2018/05/24
Type          PurRqs
DocNum    4324044999
Pos          00010 *
Change             3
Avail              6
Name: 4, dtype: object
Partnr        544011
Date      2018/05/24
Type           Order
DocNum    5236289731
Pos           000050
Change            -1
Avail              5
Nam

Name: 5, dtype: object
Partnr         19145
Date      2018/05/16
Type           Order
DocNum    5236290321
Pos           000060
Change            -4
Avail             30
Name: 6, dtype: object
Partnr         19145
Date      2018/05/17
Type           Order
DocNum    5236289007
Pos           000400
Change            -5
Avail             25
Name: 7, dtype: object
Partnr         19145
Date      2018/05/31
Type          OrdRes
DocNum             0
Pos                0
Change           -10
Avail             15
Name: 8, dtype: object
Partnr         19145
Date      2018/06/11
Type          PurRqs
DocNum    4324039195
Pos          00010 *
Change           147
Avail            162
Name: 9, dtype: object
Partnr         19145
Date      2018/06/11
Type          PurRqs
DocNum    4324045413
Pos          00010 *
Change           147
Avail            309
Name: 10, dtype: object
Partnr         19145
Date      2018/08/06
Type          PurRqs
DocNum    4324034632
Pos          00010 *
Change           147


Name: 40, dtype: object
Partnr       8001413
Date      2019/12/02
Type          IndReq
DocNum             0
Pos                0
Change            -9
Avail              0
Name: 41, dtype: object
Partnr       8001413
Date      2020/01/02
Type          PurRqs
DocNum    4315635704
Pos            00010
Change             9
Avail              9
Name: 42, dtype: object
Partnr       8001413
Date      2020/01/02
Type          IndReq
DocNum             0
Pos                0
Change            -9
Avail              0
Name: 43, dtype: object
Partnr       8001413
Date      2020/02/03
Type          PurRqs
DocNum    4315635705
Pos            00010
Change             9
Avail              9
Name: 44, dtype: object
Partnr       8001413
Date      2020/02/03
Type          IndReq
DocNum             0
Pos                0
Change            -9
Avail              0
Name: 45, dtype: object
Partnr       8001413
Date      2020/03/02
Type          PurRqs
DocNum    4315635706
Pos            00010
Change          

In [22]:
len(total)

38

In [ ]:
total[0]

In [ ]:
total[1]

In [ ]:
total[2]

In [ ]:
total[3]

In [ ]:
total[4]

In [ ]:
total[5]

In [ ]:
total[6]

In [ ]:
total[7]

In [23]:
final=pd.DataFrame()
for i in total : 
        final=final.append(i)

In [24]:
#To save excel file 
writer=pd.ExcelWriter('output.xlsx')
final.to_excel(writer,'Sheet1')
writer.save()